In [1]:
import IPython
import numpy as np
import matplotlib.pyplot as plt
import cffi

In [2]:
sr = 12000 # sample rate
ft = 442*4 # tone freq

In [3]:
tone = np.sin(2*np.pi*ft/sr*np.arange(sr))

In [4]:
tone_os = np.zeros((tone.shape[0]*4), dtype=float)

In [5]:
tone_os[0::4] = tone

In [ ]:
fig, ax = plt.subplots()
ax.plot(tone_os[:40])

In [ ]:
IPython.display.Audio(tone_os, rate=sr*4)

In [6]:
!g++ -shared -fPIC -o n005.so n005.cc

In [7]:
ffi = cffi.FFI()

In [8]:
ffi.cdef("""
  typedef struct {
    double   *buf;
    uint64_t  buf_len;
    double   *coeffs;
    uint64_t  coeffs_len;
    int64_t   inow;
  } Fir;
  double _ZN3Fir4nextEd(Fir *, double);
  extern Fir mvav2, mvav4;
  extern double mvav2_buf[2];
  extern double mvav2_coeffs[2];
  extern double mvav4_buf[4];
  extern double mvav4_coeffs[4];
""")

In [9]:
C = ffi.dlopen("./n005.so")

In [10]:
dir(C)

['_ZN3Fir4nextEd',
 'mvav2',
 'mvav2_buf',
 'mvav2_coeffs',
 'mvav4',
 'mvav4_buf',
 'mvav4_coeffs']

In [11]:
C.mvav2

<cdata 'Fir &' 0x7f8d802b0080>

In [12]:
C.mvav2_coeffs

<cdata 'double[2]' 0x7f8d802b0040>

In [13]:
dir(C.mvav2)

['buf', 'buf_len', 'coeffs', 'coeffs_len', 'inow']

In [14]:
tone_os_fil2 = [C._ZN3Fir4nextEd(ffi.addressof(C.mvav2), x) for x in tone_os]

In [15]:
tone_os_fil2[:10]

[0.0,
 0.0,
 0.0,
 0.0,
 0.3995277307681099,
 0.3995277307681099,
 0.0,
 0.0,
 0.4804379483484493,
 0.4804379483484493]

In [ ]:
IPython.display.Audio(tone_os_fil2, rate=sr*4)

In [16]:
tone_os_fil4 = [C._ZN3Fir4nextEd(ffi.addressof(C.mvav4), x) for x in tone_os]

In [17]:
tone_os_fil4[:10]

[0.0,
 0.0,
 0.0,
 0.0,
 0.19976386538405494,
 0.19976386538405494,
 0.19976386538405494,
 0.19976386538405494,
 0.24021897417422464,
 0.24021897417422464]

In [ ]:
IPython.display.Audio(tone_os_fil4, rate=sr*4)